In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf

def shape_change( y):
    y = (np.arange(10) == y).astype(np.float32)
    return y

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
#x_train, x_test = x_train, x_test
copy_x_test = x_test[0:11]
copy_y_test = y_test[0:11]
x_train, x_test = x_train/255.0, x_test/255.0
y_train = shape_change( y_train)
y_test = shape_change( y_test)



In [2]:
regularization_method = 2 #0 nic | 2 L2
epochs = 100
layers = 8 # 2 | 5 | 10
optimalization_method = 0 #0 Adam | 1 Adagrad | 2 Adadelta 
train_size = 50000
test_size = 10000

x_train = x_train[:train_size]
y_train = y_train[:train_size]
x_test = x_test[:test_size]
y_test = y_test[:test_size]


learning_rate = 0.001
batch_size = 1000
show_steps = 1



In [3]:
if layers == 8:

    X = tf.placeholder(tf.float32, shape=(None, 32,32,3))#(batch_size, 3072))
    Y = tf.placeholder(tf.float32, shape=(None, 10))#(batch_size, 10))
    
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))#[x*y in out]
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 64], mean=0, stddev=0.08))#[x*y in out]
    # 222 wielkość filtra [filter_height, filter_width, in_channels, out_channels] 

    def Deep_Network(data):
        
        conv1 = tf.nn.conv2d(data, conv1_filter, strides=[1,1,1,1], padding='SAME')
        conv1 = tf.nn.relu(conv1)
        conv1_pool = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
        # 111 mierzenie pooling 1)zmiana padding na valid | zmiana ksize na większy | zmiana strides
        conv1_batch = tf.layers.batch_normalization(conv1_pool)
        
        conv2 = tf.nn.conv2d(conv1_batch, conv2_filter, strides=[1,1,1,1], padding='SAME')
        conv2 = tf.nn.relu(conv2)
        conv2_pool = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')    
        conv2_batch = tf.layers.batch_normalization(conv2_pool)
        
        flat = tf.contrib.layers.flatten(conv2_batch)
        
        full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=384, activation_fn=tf.nn.relu)
        full1_batch = tf.layers.batch_normalization(full1)
        full2 = tf.contrib.layers.fully_connected(inputs=full1_batch, num_outputs=192, activation_fn=tf.nn.relu)
        out_layer = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=tf.nn.softmax)


       
        return out_layer


In [4]:
#########################################################################################
logits = Deep_Network(X)


if regularization_method == 0 :   
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    
if regularization_method == 2 :
    variabl = tf.trainable_variables() 
    lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in variabl if 'w' in v.name ]) * 0.001
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y) + lossL2)
 

if optimalization_method == 0 :
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
if optimalization_method == 1 :
    optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
if optimalization_method == 2 :
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)


init = tf.global_variables_initializer()

pred = tf.nn.softmax(logits, name="predictions")
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
a1 = tf.argmax(Y,1)
a2 = tf.argmax(logits,1)
confusion_matrix = tf.confusion_matrix(labels = a1, predictions = a2)


#tf.summary.histogram("predictions", pred) #new

wykres_acc1 = []
cost_list = []
#counter = 0
with tf.Session() as sess:
    sess.run(init)
    #train_writer = tf.summary.FileWriter( './logs/1/train ', sess.graph)#new
    for epoch in range(epochs):
        avg_cost = 0.0
        total_batch = int(train_size/batch_size)

        for i in range(total_batch): #trenowanie batchy
            batch_x, batch_y = x_train[batch_size * i : batch_size * (i + 1)], y_train[batch_size * i : batch_size * (i + 1)]
            #counter +=1
            #merge = tf.summary.merge_all()
            #summary,_, c = sess.run( [merge, train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
            avg_cost += c / total_batch #koszt
            #train_writer.add_summary(summary, counter)

        if epoch % show_steps == 0:
            
            print("Epoch:", (epoch+1), "cost={:.2f}".format(avg_cost))
            cost_list.append(avg_cost)
            acc1 = accuracy.eval({X: x_test[:5001], Y: y_test[:5001]})
            acc2 = accuracy.eval({X: x_test[5001:10000], Y: y_test[5001:10000]})

            wykres_acc1.append(acc1)

            print("Accuracy1:", acc1)
            print("Accuracy2:", acc2)
            print("Accuracy valid:", ((acc2+acc1)/2) )
            #tf.summary.histogram("accuracy", acc1) #new

    #acc1 = accuracy.eval({X: x_test, Y: y_test})
    #wykres_acc1.append(acc1)
    #print("End accuracy:", acc1)
    
   
    
    #print(confusion_matrix.eval({X: x_test, Y: y_test}))

    print(logits)
    print(Y)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 1 cost=2.41
Accuracy1: 0.38372326
Accuracy2: 0.37467495
Accuracy valid: 0.3791990876197815
Epoch: 2 cost=2.14
Accuracy1: 0.47990403
Accuracy2: 0.4708942
Accuracy valid: 0.47539910674095154
Epoch: 3 cost=2.06
Accuracy1: 0.5280944
Accuracy2: 0.5181036
Accuracy valid: 0.5230990052223206
Epoch: 4 cost=2.01
Accuracy1: 0.5632873
Accuracy2: 0.54530907
Accuracy valid: 0.5542981624603271
Epoch: 5 cost=1.99
Accuracy1: 0.57788444
Accuracy2: 0.56471294
Accuracy valid: 0.5712987184524536
Epoch: 6 cost=1.96
Accuracy1: 0.60008
Accuracy2: 0.58311665
Accuracy valid: 0.5915983319282532
Epoch: 7 cost=1.93
Accuracy1: 0.61527693
Accuracy2: 0.6037207
Accuracy valid: 0.6094988584518433
Epoch: 8 cost=1.91
Accuracy1: 0.6310738
Accuracy2: 0.6189238
Accuracy valid: 0.6249988079071045
Epoch: 9 cost=1.91
Accuracy1: 0.63

Epoch: 82 cost=1.66
Accuracy1: 0.72785443
Accuracy2: 0.7325465
Accuracy valid: 0.730200469493866
Epoch: 83 cost=1.65
Accuracy1: 0.7376525
Accuracy2: 0.74034804
Accuracy valid: 0.7390002608299255
Epoch: 84 cost=1.66
Accuracy1: 0.74825037
Accuracy2: 0.7555511
Accuracy valid: 0.7519007325172424
Epoch: 85 cost=1.65
Accuracy1: 0.74725056
Accuracy2: 0.74814963
Accuracy valid: 0.7477000951766968
Epoch: 86 cost=1.64
Accuracy1: 0.7458508
Accuracy2: 0.74894977
Accuracy valid: 0.7474002838134766
Epoch: 87 cost=1.63
Accuracy1: 0.7318536
Accuracy2: 0.74194837
Accuracy valid: 0.7369009852409363
Epoch: 88 cost=1.63
Accuracy1: 0.7282544
Accuracy2: 0.7311462
Accuracy valid: 0.7297003269195557
Epoch: 89 cost=1.63
Accuracy1: 0.7324535
Accuracy2: 0.74274856
Accuracy valid: 0.7376010417938232
Epoch: 90 cost=1.64
Accuracy1: 0.7376525
Accuracy2: 0.74674934
Accuracy valid: 0.7422009110450745
Epoch: 91 cost=1.64
Accuracy1: 0.7344531
Accuracy2: 0.74054813
Accuracy valid: 0.7375006079673767
Epoch: 92 cost=1.64
A

In [ ]:
'''%matplotlib inline
import pylab
x = list(range(1, epoch+3))
y = wykres_acc1
pylab.ylabel('Accuracy')
pylab.plot(x,y,'bo', label='')
pylab.show()
'''

In [ ]:
'''%matplotlib inline
import pylab
x = list(range(1, epoch+2 ) )
y = cost_list
pylab.xlabel('Epoch')
pylab.ylabel('Cost')
pylab.plot(x,y,'bo', label='')
pylab.show()
'''